<a href="https://colab.research.google.com/github/ansar2019/image-captionig/blob/main/PaliGemma_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install huggingface_hub
import os
from huggingface_hub import login
from transformers import (
    PaliGemmaProcessor,
    PaliGemmaForConditionalGeneration,
)
from transformers.image_utils import load_image
import torch

# Log in to Hugging Face using your token
# Check if the environment variable HUGGINGFACE_TOKEN is set, otherwise prompt for the token
token = os.environ.get("HUGGINGFACE_TOKEN")
if token:
    login(token=token)
else:
    login()

model_id = "google/paligemma2-3b-pt-224"

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg"
image = load_image(url)

try:
  model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto").eval()
  processor = PaliGemmaProcessor.from_pretrained(model_id)

  # Leaving the prompt blank for pre-trained models
  prompt = ""
  model_inputs = processor(text=prompt, images=image, return_tensors="pt").to(torch.bfloat16).to(model.device)
  input_len = model_inputs["input_ids"].shape[-1]

  with torch.inference_mode():
      generation = model.generate(**model_inputs, max_new_tokens=100, do_sample=False)
      generation = generation[0][input_len:]
      decoded = processor.decode(generation, skip_special_tokens=True)
      print(decoded)

except Exception as e:
  print(f"An error occurred: {e}")
  print("Please make sure you have requested access to the model and are logged in with a valid token.")

In [12]:
!unzip "/content/t3.zip"

In [16]:
!unzip "/content/t2.zip""

In [21]:
import glob
from PIL import Image
import torch
from transformers import (
    PaliGemmaProcessor,
    PaliGemmaForConditionalGeneration,
)


filenames = glob.glob("/content/t2/*.jpg")
filenames.sort()
# Load the model and processor outside the loop to prevent repeated loading
model_id = "google/paligemma2-3b-pt-224" #Added model_id since it is in other cells
try:
  model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto").eval()
  processor = PaliGemmaProcessor.from_pretrained(model_id)
  for image in filenames:
    raw_image = Image.open(image)
  # Leaving the prompt blank for pre-trained models
    prompt = ""
    model_inputs = processor(text=prompt, images=raw_image, return_tensors="pt").to(torch.bfloat16).to(model.device)
    input_len = model_inputs["input_ids"].shape[-1]

    with torch.inference_mode():
        generation = model.generate(**model_inputs, max_new_tokens=100, do_sample=False)
        generation = generation[0][input_len:]
        decoded = processor.decode(generation, skip_special_tokens=True)
    print(decoded)
    with open('caption', 'a') as fp:
          fp.write(str([image])+str([decoded]))
          fp.write('\n')
except Exception as e:
    print(f"An error occurred: {e}")
    print("Please make sure you have requested access to the model on Hugging Face and are logged in with a valid token.")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')